In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
treatsdf = pd.read_csv('../data/treatscleaned_final.csv')
ingcount = pd.read_csv('../data/ingredientscount.csv')

In [ ]:
treatsdf.head()

In [ ]:
ingcount.head()

In [ ]:
treatsdf.head()

In [ ]:
#creating a column identifying whether or not a treat is limited ingredient.
#there is no regulatory meaning of 'Limited Ingredient', so 8 is selected as the cutoff based on the
#maximum number of ingredients present in our existing homemade dog treat recipes, 8 also represents the 25%ile.

limited_ing = []

for x in treatsdf['ing_count']:
    if x <= 8:
        limited_ing.append('Yes')
    else:
        limited_ing.append('No')

In [ ]:
treatsdf['limited_ing'] = limited_ing

In [ ]:
treatsdf.describe()

In [ ]:
treatsdf.corr()

In [ ]:
#plotting dog treat price against rating.

sns.scatterplot(x = treatsdf['rating'], y = treatsdf['price'])
plt.xlabel("Rating")
plt.ylabel("Price")
plt.title("Do lower cost treats rate higher?")
plt.show()

Price and rating relationship seems to imply that lower cost treats typically rate higher. Calculated correlation coeffecient doesn't mathematically support this, however.

Research paper "Price Effects in Online Product Reviews: An Analytical Model and Empirical Analysis." appears to confirm that uni-dimensional rating systems are substantially biased by price, where rating is based more on 'perceived value' over 'perceived quality'.

Li, Xinxin, and Lorin M. Hitt. “Price Effects in Online Product Reviews: An Analytical Model and Empirical Analysis.” MIS Quarterly, vol. 34, no. 4, 2010, pp. 809–31. JSTOR, https://doi.org/10.2307/25750706. Accessed 14 June 2023.

In [ ]:
#plotting ingredient count against price.

sns.scatterplot(x = treatsdf['ing_count'], y = treatsdf['price'], size = treatsdf['rating'], hue = treatsdf['rating'], 
                palette = 'viridis')
plt.xlabel('Ingredient Count')
plt.ylabel('Price')
plt.title("Are limited ingredient treats more expensive?")
#plotting a line at 8 or my cutoff for 'limited ingredient'
plt.axvline(x = 8, linestyle = ':', color = 'orange')
plt.legend(title = "Rating", bbox_to_anchor = (1.0, 1.0))
plt.show()

The most expensive treats are considered 'Limited Ingredient'. It doesn't appear to directly correlate, with a calculated correlation coeffecient of -.17. The vertical line represents my cutoff for 'limited ingredient' treats.

In [ ]:
#calculating cost per ounce for treats

cost_per_oz = treatsdf['price'] / treatsdf['size']

In [ ]:
#plotting distribution of cost per ounce

sns.boxplot(x = cost_per_oz, showfliers = True, palette = 'viridis', showmeans=True)
plt.title('Dog Treat Price Per Ounce')
plt.show()

While I made an effort to convert all measures to ounces, few are by count. I believe this is resulting in the outliers. Weight data was not available from the website after further investigation.

In [ ]:
#plotting distribution of cost per ounce, removed outliers for a cleaner presentation.

sns.boxplot(x = cost_per_oz, showfliers = False, palette = 'viridis', showmeans=True)
plt.title('Dog Treat Price Per Ounce')
plt.show()

In [ ]:
#basics stats relative to cost per ounce.

cost_per_oz.describe()

In [ ]:
#quick math pricing an 8 ounce product based on median and mean cost per ounce.

print('The price of an 8 oz product based on median cost per ounce is ' + str(.8325 * 8) + '.')
print('The price of an 8 oz product based on mean cost per ounce is ' + str(2.561517 * 8) + '.')

With the number of outliers present, median cost may be a more accurate measure to consider for pricing of a new product.

In [ ]:
#understanding common pack sizes

treatsdf['size'].describe()

50%ile for product size is 8 ounces. 